In [2]:
# Import packages
import numpy as np
import pandas as pd
import cvxpy as cp
import mosek
import matplotlib.pyplot as plt
import scipy.stats
import phi_divergence as phi
import time
import math
import random

In [2]:
# Matplotlib settings:
plt.rcParams['figure.figsize'] = [9, 7]
plt.rcParams['figure.dpi'] = 100 # can be increased for better quality

The problem we examine is as follows:

\begin{align}
\label{math_form:examples:pm2}
    \max_{\mathbf{x}}&~\theta \\
    \text{s.t.}&~\mathbf{r}^T \mathbf{x} \geq \theta \\
    &~\mathbf{e}^T \mathbf{x} = 1, \\
    &~\mathbf{x} \geq 0,
\end{align}

where $\mathbf{x}, \mathbf{r} \in \mathbb{R}^{k}$

We randomly sample $N$ returns for k assets, which is done in the following way:

\begin{equation}
\tilde{r}_{i}=\left\{\begin{array}{ll}
\frac{\sqrt{\left(1-\gamma_{i}\right) \gamma_{i}}}{\gamma_{i}} & \text { with probability } \gamma_{i} \\[2mm]
-\frac{\sqrt{\left(1-\gamma_{i}\right) \gamma_{i}}}{1-\gamma_{i}} & \text { with probability } 1-\gamma_{i}
\end{array}, \quad \gamma_{i}=\frac{1}{2}\left(1+\frac{i}{k + 1}\right), \quad i=1, \ldots, k. \right.
\end{equation}

In [38]:
# Problem specific functions:
def generate_data(k, N):
    np.random.seed(1)
    gamma = np.fromiter(((1/2)*(1 + (i/(k+1))) for i in range(1,k+1)), float)
    return_pos = np.fromiter(((math.sqrt((1-gamma[i])*gamma[i])/gamma[i]) for i in range(0,k)), float)
    return_neg = np.fromiter((-(math.sqrt((1-gamma[i])*gamma[i])/(1-gamma[i])) for i in range(0,k)), float)
    returns = np.empty([N,k])
    for n in range(0, N):
        for i in range(0, k):
            prob = np.random.uniform()
            if prob <= gamma[i]:
                returns[n, i] = return_pos[i]
            else:
                returns[n, i] = return_neg[i]
    return returns 

def solve_problem(k, S, time_limit):
    x = cp.Variable(k, nonneg = True)
    theta = cp.Variable(1)
    constraints = [theta - (S @ x) <= 0, cp.sum(x) == 1]
    obj = cp.Maximize(theta)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK, mosek_params = {mosek.dparam.optimizer_max_time: time_limit})
    x_value = np.concatenate((theta.value,x.value)) # Combine x and theta into 1 single solution vector
    return(x_value, prob.value)

def uncertain_constraint(S, x):
    return x[0] - (S @ x[1:]) # Assume that x[0] contains theta variable 

In [39]:
k = 5
N = 10

In [40]:
returns = generate_data(k, N)
returns

array([[ 0.84515425, -1.41421356,  0.57735027,  0.4472136 ,  0.30151134],
       [ 0.84515425,  0.70710678,  0.57735027,  0.4472136 ,  0.30151134],
       [ 0.84515425, -1.41421356,  0.57735027, -2.23606798,  0.30151134],
       [-1.18321596,  0.70710678,  0.57735027,  0.4472136 ,  0.30151134],
       [-1.18321596, -1.41421356,  0.57735027,  0.4472136 ,  0.30151134],
       [-1.18321596,  0.70710678,  0.57735027,  0.4472136 ,  0.30151134],
       [ 0.84515425,  0.70710678, -1.73205081,  0.4472136 ,  0.30151134],
       [ 0.84515425, -1.41421356, -1.73205081,  0.4472136 ,  0.30151134],
       [-1.18321596, -1.41421356,  0.57735027,  0.4472136 ,  0.30151134],
       [ 0.84515425, -1.41421356,  0.57735027,  0.4472136 ,  0.30151134]])

In [41]:
x, obj = solve_problem(k, returns, 60)

In [42]:
x

array([3.01511345e-01, 7.34484430e-13, 1.45647885e-11, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00])

In [44]:
x[0] - returns @ x[1:]

array([ 2.65815148e-12, -2.82384671e-11,  2.65815148e-12, -2.67486588e-11,
        4.14795975e-12, -2.67486588e-11, -2.82384671e-11,  2.65815148e-12,
        4.14795975e-12,  2.65815148e-12])

In [4]:
# Methodology-related functions:
def lowbound(p, r, par, phi_div):
    q = cp.Variable(2, nonneg = True)
    constraints = [cp.sum(q) == 1]
    constraints = phi_div(p,q,r,par,constraints)
    obj = cp.Minimize(q[0])
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(prob.value)

def add_scenarios(add_strategy, data, Z_values, Z_indices, constr, vio, beta, lb):
    ind = pick_scenarios_to_add(add_strategy, len(data), constr, vio, beta, lb)
    Z_indices.append(ind)
    scen_to_add = np.array([data[ind]])
    Z_values = np.append(Z_values, scen_to_add, axis = 0)
    return Z_values, Z_indices

def pick_scenarios_to_add(add_strategy, N, constr, vio, beta, lb):
    if add_strategy == 'smallest_vio':   # the least violated scenario is added   
        return np.where(constr == np.min(vio))[0][0]
    elif add_strategy == 'random_vio':
        return np.random.choice(np.where(constr > (0+numeric_precision))[0])
    elif add_strategy == 'N*(beta-lb)_smallest_vio':   # the N*(beta-lb)-th scenario is added
        rank = np.ceil(N*(beta-lb)).astype(int)
        if rank > len(vio):
            return np.where(constr == np.max(vio))[0][0]
        vio_sort = np.sort(vio) 
        vio_value = vio_sort[rank-1]     # -1 to correct for python indexing
        return np.where(constr == vio_value)[0][0]
    elif add_strategy == 'random_weighted_vio':
        rank = np.ceil(N*(beta-lb)).astype(int)
        if rank > len(vio) or rank < 2:
            return np.where(constr == np.max(vio))[0][0]
        vio_sort = np.sort(vio)  
        vio_ideal = (vio_sort[rank-1] + vio_sort[rank-2]) / 2    # -1 to correct for python indexing
        weights = [(1 / (abs(vio_ideal - i))) for i in vio]
        sum_weights = sum(weights)
        probs = [i/sum_weights for i in weights]
        ind = np.random.choice(a = len(vio), p = probs)  
        vio_chosen = vio[ind]
        return np.where(constr == vio_chosen)[0][0]
    else:
        print("Error: did not provide valid addition strategy")
        return None

def remove_scenarios(remove_strategy, Z_values, Z_indices, x, numeric_precision):
    if remove_strategy == 'all_inactive':
        constr_Z = Z_values.dot(x)
        set_trace()
        ind = np.where(constr_Z < (1-numeric_precision))[0]
        Z_values = np.delete(Z_values, ind, axis=0)
    elif remove_strategy == 'random_active':
        constr_Z = Z_values.dot(x)
        active = np.where(constr_Z >= (1-numeric_precision))[0]
        ind = np.random.choice(active)
        Z_values = np.delete(Z_values, ind, axis=0)
    elif remove_strategy == 'random_any':
        ind = np.random.choice(len(Z_values))
        Z_values = np.delete(Z_values, ind, axis=0)
    else:
        print("Error: did not provide valid removal strategy")
    
    if isinstance(ind, np.ndarray):
        ind_set = set(ind.flatten())
        Z_indices = [i for j, i in enumerate(Z_indices) if j not in ind_set] 
    else:
        ind = ind.item()
        del Z_indices[ind]
    
    return Z_values, Z_indices

def search_alg(data_train, beta, alpha, time_limit_search, time_limit_solve, threshold_time_solve, max_nr_solutions,
              add_strategy, remove_strategy, improve_strategy, par, phi_div, phi_dot, numeric_precision):

    # Get extra info
    N_train = len(data_train)
    k = data_train.shape[1]
    r = phi_dot/(2*N_train)*scipy.stats.chi2.ppf(1-alpha, 1)
    
    # Initialize algorithm
    start_time = time.time()
    Z_values = np.array([data_train[0]]) # Assume first index contains nominal data
    Z_indices = [0] # Tracks the indices of the scenarios in Z
    lb = -np.inf
    num_iter = {'add':0, 'remove':0, 'improve':0}
    solutions = []
    np.random.seed(1) # Set seed for random strategies
    
    while True:
        solve_start_time = time.time()
        [x, obj] = solve_problem(k, Z_values, time_limit_solve)
        solve_time = time.time() - solve_start_time
            
        # Compute the lower bound on training data (to get a feel for feasibility)
        constr = data_train.dot(x)
        vio = constr[constr>(1+numeric_precision)]   
        p_vio = len(vio)/N_train
        p = np.array([1-p_vio, p_vio])
        
        if p_vio == 0:
            lb = 1
        else:
            lb = lowbound(p, r, par, phi_div)
        
        solutions.append({'sol': x, 'obj': obj, 'time': (time.time()-start_time), 
                          'lb_train': lb, 'lb_test': np.nan, 'scenario_set': Z_indices.copy()})
                
        if len(solutions) >= max_nr_solutions:
            break
        
        if solve_time >= threshold_time_solve and len(Z_values) > 1: # Invoke removal scenarios (to improve solve efficiency)
            Z_values, Z_indices = remove_scenarios(remove_strategy, Z_values, Z_indices, x, numeric_precision)
            num_iter['remove'] += 1
        elif lb >= beta: # have achieved feasibility, now we remove some scenarios
            Z_values, Z_indices = remove_scenarios(improve_strategy, Z_values, Z_indices, x, numeric_precision)
            num_iter['improve'] += 1
        else: # Add scenario if lb still lower than beta
            Z_values, Z_indices = add_scenarios(add_strategy, data_train, Z_values, Z_indices, constr, vio, beta, lb) 
            num_iter['add'] += 1
        
        if (time.time()-start_time) >= time_limit_search:
            break   
    
    runtime = time.time() - start_time
    return runtime, num_iter, solutions  
    
def evaluate_alg(solutions, data_test, beta, alpha, par, phi_div, phi_dot, numeric_precision):
    start_time = time.time()
    
    # Get extra info
    N_test = len(data_test)
    k = data_test.shape[1]
    r = phi_dot/(2*N_test)*scipy.stats.chi2.ppf(1-alpha, 1)
    
    # Store best solution info
    best_sol = {'sol': None}
    
    for sol_info in solutions:
        x = sol_info['sol']
        
        # Evaluate "true" lb on test data
        constr_test = data_test.dot(x)
        vio_test = constr_test[constr_test>(1+numeric_precision)]   
        p_vio = len(vio_test)/N_test
        p = np.array([1-p_vio, p_vio])
        
        if p_vio == 0:
            lb = 1
        else:
            lb = lowbound(p, r, par, phi_div)

        sol_info['lb_test'] = lb
        
        if best_sol['sol'] is None or (best_sol['lb_test'] < beta and lb > best_sol['lb_test']):
            best_sol = sol_info
        elif ((lb >= beta and sol_info['obj'] > best_sol['obj']) 
              or (lb > best_sol['lb_test'] and sol_info['obj'] >= best_sol['obj'])):
            best_sol = sol_info
    
    runtime = time.time() - start_time
    return runtime, best_sol

In [23]:
# Auxillary functions:
def check_conv_comb(Z_arr):
    conv_comb_points = []
    if len(Z_arr) >= 3:
        for i in range(len(Z_arr)):
            z_i = Z_arr[i]
            Z_rest = np.append(Z_arr[:i], Z_arr[(i+1):], axis = 0)
            # solve optimization problem, if feasible, z_i is convex combination of points in Z_rest 
            # (https://en.wikipedia.org/wiki/Convex_combination)
            alpha = cp.Variable(len(Z_rest), nonneg = True)
            constraints = [alpha @ Z_rest == z_i, cp.sum(alpha) == 1]
            obj = cp.Maximize(alpha[0]) # no true objective function, only interested whether feasible solution exists
            prob = cp.Problem(obj,constraints)
            prob.solve(solver=cp.MOSEK)
            if prob.status != 'infeasible':
                conv_comb_points.append(i) 
    return conv_comb_points

def compute_calafiore_N_min(dim_x, beta, alpha):
    N_min = np.ceil(dim_x /((1-beta)*alpha)).astype(int) - 1
    return N_min

def compute_alamo_N_min(dim_x, beta, alpha):
    N_min = (math.exp(1) / (math.exp(1) - 1)) * (1 / (1-beta)) * (dim_x + math.log(1/alpha))
    N_min = np.ceil(N_min).astype(int) 
    return N_min
                   
def compute_calafiore_vio_bound(N, dim_x, beta):
    bound = math.comb(N, dim_x) * ((1 - (1-beta))**(N - dim_x))
    return bound

def determine_calafiore_N_min(dim_x, beta, alpha):
    # Do bisection search between 1 and calafiore N_min
    a = 1
    f_a = compute_calafiore_vio_bound(a, dim_x, beta)
    b = compute_calafiore_N_min(dim_x, beta, alpha)
    f_b = compute_calafiore_vio_bound(b, dim_x, beta)
    while True:
        c = np.ceil((a+b)/2).astype(int)
        f_c = compute_calafiore_vio_bound(c, dim_x, beta)
        if abs(a-b) == 1:
            if f_c <= alpha:
                return c
            else:
                return c-1
        if f_c > alpha:
            a = c
            f_a = f_c
        else:
            b = c
            f_b = f_c

def compute_campi_vio_bound(N, dim_x, beta):
    bound = 0
    for i in range(dim_x):
        try:
            bound += math.comb(N, i) * ((1-beta)**i) * (1 - (1-beta))**(N - i)
        except OverflowError as e:
            print("Note: Overflow error in computing Campi bound, will return Alamo bound instead")
            return -1
    return bound

            
def determine_campi_N_min(dim_x, beta, alpha):
    # Do bisection search between 1 and calafiore N_min
    a = 1
    f_a = compute_campi_vio_bound(a, dim_x, beta)
    b = compute_alamo_N_min(dim_x, beta, alpha)
    f_b = compute_campi_vio_bound(b, dim_x, beta)
    if f_b == -1: # To catch overflow error with computing bounds for large k
        return b
    
    while True:
        c = np.ceil((a+b)/2).astype(int)
        f_c = compute_campi_vio_bound(c, dim_x, beta)   
        if abs(a-b) == 1:
            if f_c <= alpha:
                return c
            else:
                return c-1
        if f_c > alpha:
            a = c
            f_a = f_c
        else:
            b = c
            f_b = f_c
            
def solve_with_campi_N(alpha, beta, numeric_precision, data, time_limit_solve):  
    start_time = time.time()
    [x, obj] = solve_problem(k, data, time_limit_solve)
    runtime = time.time() - start_time
    true_prob = get_true_prob(x, k)
    return runtime, x, obj, true_prob, data
    